# GEOGEAR
This tool provides an easy-to-use method for generating integrated data products from disparate spatial data. 
* A graphical user interface allows the user to include spatial data of different file types and customize the requirements for the integrated data product.
* Parallelization of compute-intensive steps and the use of distributed remote cloud computing, computational efficiency is achieved, even when large volume spatial data is used. 
* Provenance information on the execution of the tool and the generation of data products across distributed workflows is generated through the use of the PROV-O ontology and ISO-19115 metadata standard.

In this notebook, the user first selects parameters (mask, projection, cell size) to create a grid that serves as a template for the integrated data product and the following functions. After that, the user selects combinations of spatial analysis functions and spatial layers that will be executed on the remote cloud infrastructure. The integrated data product and its provenance information will be returned to the user after completion of the analysis.

All the user interactions with this notebook should be through the widgets (no coding is required). If any widgets are not loaded (correctly), please re-run that specific cell or re-run the whole notebook by pressing `>>` in the toolbar.

This installment of GEOGEAR is still under development and can therefore still contain bugs. If you experience any issues or have questions, make sure to contact me (Tim Lenters) at timlenters@gmail.com.

## Load prerequisites

In [1]:
import warnings
warnings.filterwarnings("ignore")
import ipywidgets as widgets
import pyproj
import geopandas as gpd
%matplotlib widget
import matplotlib.pyplot as plt
from termcolor import colored
import os
import fiona

## 1. Load mask
By clicking the "Load mask" button, a mask file can be selected from your local file directory. This can be any polygon layer (from a local to global extent), of any file format, as long as it consists of one file (i.e. ESRI Shapefiles (`.shp`)are not supported at this time, GeoPackage or GeoJSON are). 

If preferred, an example file can be used as mask by clicking "Use example mask" button. This is a GeoPackage boundary file of the Netherlands.


In [2]:
load_mask = widgets.FileUpload(
    description = "Load mask",
    multiple = False
)

example_file = widgets.Button(
    description = "Use example mask"
)

def mask_ex(_):
    global mask_ex
    load_mask._counter = 0
    load_mask.value.clear()
    mask_ex = "NL_boundary.gpkg"
    with loaded_mask:
        loaded_mask.clear_output()
        print("NL_boundary.gpkg is loaded.")

def on_mask_change(_):
    load_mask._counter = 1
    with loaded_mask:
        loaded_mask.clear_output()
        print(str(list(load_mask.value.keys())[0]) + " is loaded.")
    
example_file.on_click(mask_ex)

load_mask.observe(on_mask_change, 'value')

loaded_mask = widgets.Output()

display(widgets.HBox([load_mask,example_file]))
display(loaded_mask)

Output()

## 2. Select projection
The dropdown-selection widget consists of all available projections as defined by the [PROJ](https://proj.org/) library. Some projections will not be valid without adding additional arguments and will show an error message when selected. The box below allows the user to add additional arguments to the proj4string to define projection details (e.g. define units, viewing angle or spatial extent). Click [this link](https://proj.org/usage/quickstart.html) for additional information on how to write these strings. 

In [3]:
proj_options = [(v, "+proj="+k) for k, v in pyproj.pj_list.items()]

proj_select = widgets.Dropdown(
    options = proj_options,
    value = "+proj=eck4",
    continuous_update=False
)

proj_input = widgets.Text(
    value = proj_select.value,
    continuous_update=False
)

proj_link = widgets.link((proj_select, 'value'), (proj_input, 'value'))

proj_error = widgets.Output()

def on_proj_change(change):
    try:
        pyproj.CRS(proj_input.value)
        proj_error.clear_output()
    except pyproj.exceptions.CRSError:
        with proj_error:
            proj_error.clear_output()
            print(colored("Select other projection or add parameters!", 'red'))

proj_input.observe(on_proj_change, 'value')
    
proj_vbox = widgets.VBox([proj_select, proj_input, proj_error])
display(proj_vbox)

## 3. Select cell size

The cell size or grid resolution can be selected by using the following widgets. With the logarithmic slider, the desired order of magnitude for the grid cell size can be selected. To enter a more precise resolution, the box below the slider can be edited. The range of the slider and `unit:` depends on the selected projection, which is automatically updated.

<span style="color:red">IMPORTANT:</span> as the cloud backend is not yet fully established, I advise that a resolution <100 meters is not used at this time.

In [4]:
def update_cellsize(value):
    if pyproj.CRS(proj_input.value).axis_info[0].unit_name == 'degree':
        cellsize_slider.min = -3
        cellsize_slider.max = 3
        cellsize_slider.value = 1
        cellsize_slider.step = -4
        cellsize_input.description = "unit: degree"
    else:
        cellsize_slider.min = 2
        cellsize_slider.max = 7
        cellsize_slider.value = 10000
        cellsize_slider.step = 1
        cellsize_input.description = "unit: metre"


cellsize_slider = widgets.FloatLogSlider(
    min = 2,
    max = 7,
    value = 10000,
    step = 1
)
cellsize_input = widgets.FloatText(
    value = cellsize_slider.value,
    description = "unit: metre",
    style=dict(description_width='initial'))
proj_input.observe(update_cellsize, 'value')



cz_link = widgets.link((cellsize_slider, 'value'), (cellsize_input, 'value'))

display(cellsize_slider)
display(cellsize_input)

FloatLogSlider(value=10000.0, max=7.0, min=2.0, step=1.0)

FloatText(value=10000.0, description='unit: metre', style=DescriptionStyle(description_width='initial'))

## 4. Make grid

In this section, the selected grid parameters can be validated by plotting the grid on the mask. First, provide a username. This will be used to create a directory which holds the files you generate. After that, press "Update plot" to plot the grid. The progress bar gives an indication what is currently being done. You can interact with the plot (e.g. zooming, panning) with the toolbar on the left.

<span style="color:red">NOTE:</span> If the plot does not show after clicking the button, re-run this cell to fix the issue.

In [5]:
from functions import grid

def make_grid(mask, cellsize, progress, path):
    
    if not os.path.exists(path + '/input'): os.makedirs(path + '/input')
    if not os.path.exists(path + '/analysis'): os.makedirs(path + '/analysis')
    
    saga_cmd = "saga_cmd"
    
    
    progress.description = "Making grid..."
    progress.value += 1
    doc = grid(mask, cellsize, path).serialize()
    

    if int(mask.area) / cellsize**2 < 200000:
    
        progress.description = "Vectorizing grid..."
        progress.value += 1
        os.system(saga_cmd
                  + ' shapes_grid 3 -GRIDS ' + path + '/output/grid/grid.tif'
                  + ' -SHAPES ' + path + '/input/grid_vector.gpkg'
                  + ' -TYPE 1')
    
        return doc, True
    
    else:
        return doc, False

button = widgets.Button(description="Update plot",
                       disabled=True)
out = widgets.Output()

with out:
    fig, ax = plt.subplots()
    plt.show()

progress = widgets.IntProgress(style=dict(description_width='initial'))

def on_button_clicked(_):
    global mask_reproj, grd_json, fig, path
    
    path = os.path.join(os.getcwd(), usr_input.value)
    if not os.path.exists(path): os.makedirs(path)
    
    progress.max = 3
    if load_mask.value != {}:
        progress.max = 4
        progress.description = "Loading mask..."
        progress.value = 1
        uploaded_filename = next(iter(load_mask.value))
        content = load_mask.value[uploaded_filename]['content']
        with fiona.io.MemoryFile(content) as mask:
            with mask.open('mask.gpkg') as collection:
                mask_reproj = gpd.GeoDataFrame.from_features(collection, crs=collection.crs).dissolve().to_crs(proj_input.value)
    elif str(loaded_mask) == "Output()":
        with out:
            out.clear_output()
            print("No mask file is loaded...")
        pass
    else:
        try:
            mask_reproj = gpd.read_file("NL_boundary.gpkg").to_crs(proj_input.value)
        except (pyproj.exceptions.CRSError, StopIteration):
            with out:
                out.clear_output()
                print("Invalid projection selected...")
    try:
        cell_est = mask_reproj.area/cellsize_input.value**2
        with out:
            grd_json = make_grid(mask_reproj, cellsize_input.value, progress, path)
            progress.description = "Plotting grid..."
            progress.value += 1
            if grd_json[1]:
                grid = gpd.read_file(path + "/input/grid_vector.gpkg")
                with out:
                    plt.cla()
                    mask_reproj.plot(ax=ax)
                    grid.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=0.5)
                    plt.show()
            else:
                with out:
                    print("Vectorizing would take too much time, proceeding without plotting grid.")
                    plt.cla()
                    mask_reproj.plot(ax=ax)
                    plt.show()
            progress.value = 0
            progress.description = ""
    except NameError:
        progress.value = 0
        progress.description = ""
        
def enable_start(_):
    if usr_input.value == '':
        button.disabled = True
    else:
        button.disabled = False
    
usr_input = widgets.Text(
    placeholder = "Enter username"
)        
   
usr_input.observe(enable_start, 'value')
button.on_click(on_button_clicked)

display(widgets.HBox([usr_input, button]))
display(progress)
display(out)

IntProgress(value=0, style=ProgressStyle(description_width='initial'))

Output()

## 5. Load layers
In this section, you first select the function(s) that you want to execute. Currently, three functions can be selected. All three of these functions are based on [SAGA GIS](http://www.saga-gis.org/en/index.html) functions, a free and open-source software for geospatial analysis. 

The three functions are:
* **coverages:** calculates the proportion of each grid cell that is covered by each feature of a categorical spatial layer. The input layers can be both vector and raster formats, as long as they describe categorical features. The output data consists of rasters (i.e. GeoTIFFs) with the coverages of each feature, as well as a table (i.e. CSV) which contains all combined coverages, with the columns `cell_ID` (unique IDs for each grid cell), `feature` (the name of each layer and the specific feature) and `proportion` (the proportion of the grid cell covered by each feature). More information about the SAGA function can be found [here](http://www.saga-gis.org/saga_tool_doc/7.7.0/grid_analysis_26.html).
* **presence_absence:** can be used with point data and calculates for each grid cell if and how many points intersect with that cell. The input data is point (vector) data and its output consists of two rasters (GeoTIFFs), one for the presence/absence of points for each grid cell and the other for the count of points per grid cell. Additionally, these data are also provided as a table (CSV) with the columns `cell_ID`, `feature` (the name of each selected layer and if it is the presence/absence or count value) and `value` (the presence/absence (0 or 1) or count value). More information about the SAGA function can be found [here](http://www.saga-gis.org/saga_tool_doc/7.7.0/grid_gridding_0.html).
* **resample:** can be used to aggregate the resolution of a raster and vector layers to the selected cell size. The input layers are rasters and can represent both continuous or categorical data. For continuous data, the mean value is calculated by default as grid cell aggregate and for categorical data, the most abundant value (majority) is taken as grid cell aggregate value. This function outputs the selected layer as a raster (GeoTIFF) with the resolution of the grid and a table (CSV) containing the `cell_ID`, `feature` (the name of the layer) and `value` (the value of each grid cell). More information about the SAGA function can be found [here](http://www.saga-gis.org/saga_tool_doc/7.7.0/grid_tools_0.html).

Other than their main functionalities, each function also harmonizes the input layers. This consists of reprojecting the layers to the selected projection and converting them to the same format (e.g. rasterizing the vector layers). 

Second, you provide names of spatial layers (with their extension) and a URL to a Google Docs file. If you want to add multiple layers per function "Add row" can be clicked. After the names and URLs are filled in, click "Add / Reset" to add them to the dictionary. If you made a mistake during this, click the same button to remove that function from the dictionary. 

If you are using the example mask from section 1, the following layers can be used:

* **GloRiC_NL.gpkg:** **URL:** https://drive.google.com/file/d/1aT9B5bIN3EABWoIBgx0DmOkHoHCVbbeU/view?usp=sharing; **type:** `Polyline`. This is a subset for the Netherlands of the "Global River Classification" dataset containing the `Reach_type` variable. More information can be found [here](https://www.hydrosheds.org/page/gloric).
* **LGN5.tif:** **URL:** https://drive.google.com/file/d/1EP0rwpugKHke-Eo3O4tCWrdYuv3nNVnL/view?usp=sharing. **type:** `Raster`. This is the "Dutch Land Use" dataset. More information can be found [here](https://www.nationaalgeoregister.nl/geonetwork/srv/api/records/714c7cde-f8ed-4371-bb26-cb063e117d6e).
* **mammal_obs.gpkg** **URL:** https://drive.google.com/file/d/1ysQZFYFnS1a1d9mqDd5yH7tDwG3zV3vV/view?usp=sharing. **type:** `Point`. This is a GBIF mammal observations dataset for the Netherlands. More information can be found [here](https://doi.org/10.15468/dl.55g7u2).

It is also possible to use your own files. This can be any spatial file type (i.e. `Raster`, `Point`, `Polygon`, `Polyline`) as long as it is one file (so no `.shp`). Make sure to select "Anyone with the link" when getting the shareable link.

After adding the files, below the "Add / Reset" button, a new dictionary entry appears. Here you can validate if you added the right layers to each function and if the names and links are spelled correctly.


In [6]:
import pandas as pd
from inspect import getmembers, isfunction
import functions

## Select functions

func_lst = []
for i in [o for o in getmembers(functions) if isfunction(o[1])]:
    if i[0] not in ['setup','grid','download_layers', 'url_to_id', 'grid_statistics']:
        func_lst.append(i[0])

    
func_select = widgets.RadioButtons(
    options = func_lst
)

    
## Select layers


add_row_btn = widgets.Button(description='Add Row')
rm_row_btn = widgets.Button(description='Remove Row')

out = widgets.Output()

def add_row(_):
    file_name_box.children += (widgets.Text(),)
    file_url_box.children += (widgets.Text(),)
        
def rm_row(_):
    file_name_box.children -= (widgets.Text(),)
    file_url_box.children -= (widgets.Text(),)
    
add_row_btn.on_click(add_row)
rm_row_btn.on_click(rm_row)

df = pd.DataFrame()


## Add function

add_btn = widgets.Button(
    description = "Add / Reset"
)

output_tbl = widgets.Output()

tbl = dict()

def add_functions(b):
    global tbl
    output_tbl.clear_output()
    
    file_names = []
    for i in range(len(file_name_box.children)):
        file_names.append({file_name_box.children[i].value:
                           file_url_box.children[i].value})

    
    if file_names == [{'': ''}]:
        tbl.pop(func_select.value, None)
    else:
        tbl[func_select.value] = file_names
        
    file_name_box.children = [widgets.Text()]
    file_url_box.children = [widgets.Text()]
    
    with output_tbl:
        for x in tbl:
            print(x+":")
            for y in tbl[x]:
                print("   Name: ", list(y.keys())[0])
                print("   URL:  ", list(y.values())[0],"\n")

add_btn.on_click(add_functions)


file_name_box = widgets.VBox(children=[widgets.Text()])
file_url_box = widgets.VBox(children=[widgets.Text()])


display(widgets.HBox([widgets.VBox([widgets.Label("Select function:"), func_select]), 
                      widgets.VBox([widgets.Label("Select layers:"), widgets.HBox([widgets.VBox([widgets.Label(value="File Name:"), file_name_box, add_row_btn]), 
                                                                                   widgets.VBox([widgets.Label(value="File URL:"), file_url_box])])])]))
display(add_btn)
display(output_tbl)
#LGN5.tif: https://drive.google.com/file/d/1EP0rwpugKHke-Eo3O4tCWrdYuv3nNVnL/view?usp=sharing
#GloRiC_NL.gpkg: https://drive.google.com/file/d/1aT9B5bIN3EABWoIBgx0DmOkHoHCVbbeU/view?usp=sharing
#zip: https://drive.google.com/file/d/15LX2X1z9ehkVxZvFq1NLQqqN0cNMIbTE/view?usp=sharing
#mammals.gpkg: https://drive.google.com/file/d/1ysQZFYFnS1a1d9mqDd5yH7tDwG3zV3vV/view?usp=sharing

Button(description='Add / Reset', style=ButtonStyle())

Output()

## 6. Start analysis

As a last step, please select the file format(s) for exporting the generated provenance data. After that, the analysis can be started by pressing the "Start analysis" button. The progress will be recorded here to give an indication on the workflow execution.

After the analysis is finished, the file structure of the integrated data product is shown below.

In [9]:
from backend import analysis
from distutils.dir_util import copy_tree
from time import time
import importlib
import sys
import shutil

importlib.reload(sys.modules['backend'])
%reload_ext autoreload
%autoreload 2
from backend import analysis


import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
         
            
def start_analysis(_):
    log.clear_output()
    try:
        with log:
            analysis(cellsize=cellsize_input.value,
                     projection=proj_input.value,
                     layers_dict=tbl,
                     grd_json=grd_json[0],
                     path=path,
                     prov=exp_prov.value)
            print("Analysis completed!\n")
            print("The integrated data product consists of the following files:")
            list_files(path + "/output")

        shutil.make_archive(path, 'zip', os.path.join(path,'output'))
    except NameError:
        with log:
            log.clear_output()
            print("Create grid first!")

    
exp_prov = widgets.SelectMultiple(
    options = ["PNG","PDF","JSON","XML","RDF"]
)
    
start_btn = widgets.Button(
    description = "Start analysis",
    disabled = False
)

start_btn.on_click(start_analysis)

log = widgets.Output()

display(widgets.Label("Export provenance data as:"))
display(exp_prov)
display(start_btn)
display(log)

Label(value='Export provenance data as:')

SelectMultiple(options=('PNG', 'PDF', 'JSON', 'XML', 'RDF'), value=())

Button(description='Start analysis', style=ButtonStyle())

Output()